# The goal is to perform dimentiality reduction for future model building.
Initial steps are just to build default models, that could fit available memory
and make a submission on Kaggle.

In [1]:
#jupyter notebook --NotebookApp.max_buffer_size=35536870912 
# to run jupyter notebook using more available memory
import os, gc, pickle, scipy.sparse, time, h5py, anndata, hdf5plugin#, lightgbm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
#from sklearn.svm import SVR takes way too long
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
#import scanpy as sc
import anndata
import hdf5plugin
from sys import getsizeof
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
import joblib
import h5py
from scipy.sparse import csr_matrix
from math import sqrt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns

data_path = "/home/skovtun/Single cell/Single_cell_data"
metadata = os.path.join(data_path,"metadata.csv")

train_cite_inputs = os.path.join(data_path,"train_cite_inputs.h5")
train_cite_targets = os.path.join(data_path,"train_cite_targets.h5")
test_cite_inputs = os.path.join(data_path,"test_cite_inputs.h5")

train_multi_inputs = os.path.join(data_path,"train_multi_inputs.h5")
train_multi_targets = os.path.join(data_path,"train_multi_targets.h5")
test_multi_inputs = os.path.join(data_path,"test_multi_inputs.h5")

sample_submission = os.path.join(data_path,"sample_submission.csv")
evaluation_ids = os.path.join(data_path,"evaluation_ids.csv")

In [2]:
# Getting indexes of cell_id for train and test samples I defined in Random_sample_cells_id.
import csv

cite_train_sample = []
cite_test_sample = []
multi_train_sample = []
multi_test_sample = []
with open('cite_train_sample.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        cite_train_sample.append(row)

with open('cite_test_sample.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        cite_test_sample.append(row)

with open('multi_train_sample.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        multi_train_sample.append(row)

with open('multi_test_sample.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        multi_test_sample.append(row)
multi_train_sample = [item for sublist in multi_train_sample for item in sublist]
multi_test_sample = [item for sublist in multi_test_sample for item in sublist]

cite_train_sample = [item for sublist in cite_train_sample for item in sublist]
cite_test_sample = [item for sublist in cite_test_sample for item in sublist]


In [3]:
print(len(multi_train_sample),len(multi_test_sample), len(cite_train_sample), len(cite_test_sample))

129500 32377 95720 23931


In [56]:
# Read the h5 file and extract the dataset values and corresponding rows
def sparse_matrix_from_file( data_file_path,data_key, filter_list):
    """
    Given path to the h5 file data_file_path and list of cells filter_list, name of the data_key,
    creates a sparse
    matrix of rows in h5 file that belongs to the cells_id from the filter_list
    """
    with h5py.File(data_file_path, 'r') as h5file:
        data = h5file[data_key]['block0_values']
        rows = h5file[data_key]['axis1'][:]
    
        # Convert rows to regular strings
        rows = [item.decode('utf-8') for item in rows]
        
        # Initialize an empty list to store selected_data in chunks
        selected_data_chunks = []
        if len(filter_list) == 0:
            filter_list = rows
        # Set the chunk size (adjust as needed)
        chunk_size = 1000
        data_list = []
        indices_list = []
        indptr_list = [0]  # Initialize with 0
        # Iterate through the data in chunks
        for start_idx in range(0, len(rows), chunk_size):
            end_idx = start_idx + chunk_size
            print(start_idx, end_idx)
            
            # Create a boolean mask for the current chunk
            chunk_mask = np.isin(rows[start_idx:end_idx], filter_list)
            
            # Use the mask to select the corresponding rows from the current chunk of data
            chunk_data = data[start_idx:end_idx][chunk_mask]
            
            
            # Process the chunk, convert to CSR format
            processed_chunk_sparse = csr_matrix(chunk_data)
                    
            # Update indptr and store the data and indices
            indptr_list.extend(processed_chunk_sparse.indptr[1:] + indptr_list[-1])
            data_list.append(processed_chunk_sparse.data)
            indices_list.append(processed_chunk_sparse.indices)
        
            
        # Combine the lists into arrays
        values = np.concatenate(data_list)
        indices = np.concatenate(indices_list)
        indptr = np.array(indptr_list)
        
        # Create the final CSR sparse matrix
        sparse_matrix = csr_matrix((values, indices, indptr)) 
    return sparse_matrix

In [5]:
#Read dataset for test_multiome and conver it to sparse matrix
multi_train_sparse_matrix = sparse_matrix_from_file( train_multi_inputs,'train_multi_inputs', multi_train_sample)

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 39000
39000 40000
40000 41000
41000 42000
42000 43000
43000 44000
44000 45000
45000 46000
46000 47000
47000 48000
48000 49000
49000 50000
50000 51000
51000 52000
52000 53000
53000 54000
54000 55000
55000 56000
56000 57000
57000 58000
58000 59000
59000 60000
60000 61000
61000 62000
62000 63000
63000 64000
64000 65000
65000 66000
66000 67000
67000 68000
68000 69000
69000 70000
70000 71000
71000 72000
72000 73000
73000 74000
74000 75000
75000 76000
76000 77000
77000 78000
78000 79000
79000 80000
80000 81000
81000 82000
82000 83000
83000 84000
84000 85000
85

In [3]:
# Read the h5 file and extract the dataset values and corresponding rows
filter_list  = multi_train_sample
with h5py.File(train_multi_inputs, 'r') as h5file:
    data = h5file['train_multi_inputs']['block0_values']
    rows = h5file['train_multi_inputs']['axis1'][:]

    # Convert rows to regular strings
    rows = [item.decode('utf-8') for item in rows]
    
    # Initialize an empty list to store selected_data in chunks
    selected_data_chunks = []
    
    # Set the chunk size (adjust as needed)
    chunk_size = 1000
    data_list = []
    indices_list = []
    indptr_list = [0]  # Initialize with 0
    # Iterate through the data in chunks
    for start_idx in range(0, len(rows), chunk_size):
        end_idx = start_idx + chunk_size
        print(start_idx, end_idx)
        
        # Create a boolean mask for the current chunk
        chunk_mask = np.isin(rows[start_idx:end_idx], filter_list)
        
        # Use the mask to select the corresponding rows from the current chunk of data
        chunk_data = data[start_idx:end_idx][chunk_mask]
        
        
        # Process the chunk, convert to CSR format
        processed_chunk_sparse = csr_matrix(chunk_data)
                
        # Update indptr and store the data and indices
        indptr_list.extend(processed_chunk_sparse.indptr[1:] + indptr_list[-1])
        data_list.append(processed_chunk_sparse.data)
        indices_list.append(processed_chunk_sparse.indices)
    
        
    # Combine the lists into arrays
    values = np.concatenate(data_list)
    indices = np.concatenate(indices_list)
    indptr = np.array(indptr_list)
    
# Create the final CSR sparse matrix
multi_train_sparse_matrix = csr_matrix((values, indices, indptr))    
        


0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 39000
39000 40000
40000 41000
41000 42000
42000 43000
43000 44000
44000 45000
45000 46000
46000 47000
47000 48000
48000 49000
49000 50000
50000 51000
51000 52000
52000 53000
53000 54000
54000 55000
55000 56000
56000 57000
57000 58000
58000 59000
59000 60000
60000 61000
61000 62000
62000 63000
63000 64000
64000 65000
65000 66000
66000 67000
67000 68000
68000 69000
69000 70000
70000 71000
71000 72000
72000 73000
73000 74000
74000 75000
75000 76000
76000 77000
77000 78000
78000 79000
79000 80000
80000 81000
81000 82000
82000 83000
83000 84000
84000 85000
85

In [7]:
multi_train_sparse_matrix.shape

(84548, 228942)

In [5]:
#TruncatedSVD
# 6000 components explains ~70% of variance
# from sklearn.decomposition import TruncatedSVD
# start_time = time.time()
# svd_multi = TruncatedSVD(n_components=150, n_iter=7, random_state=42)
# svd_multi.fit(multi_train_sparse_matrix)
# print(svd_multi.explained_variance_ratio_.sum())
# print(f"--- {time.time() - start_time} seconds ---")
# 0.012246355
# --- 800.2067103385925 seconds ---


0.012246355
--- 800.2067103385925 seconds ---


In [6]:
#TruncatedSVD
# 6000 components explains ~70% of variance
from sklearn.decomposition import TruncatedSVD
start_time = time.time()
svd_multi_500 = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
svd_multi_500.fit(multi_train_sparse_matrix)
print(svd_multi_500.explained_variance_ratio_.sum())
print(f"--- {time.time() - start_time} seconds ---")
#0.02362837
#--- 1864.4796884059906 seconds ---

0.02362837
--- 1747.8000128269196 seconds ---


In [7]:
multi_train_500 = svd_multi_500.transform(multi_train_sparse_matrix)

In [12]:
multi_train_500.shape

(84548, 500)

In [6]:
# from scipy.sparse.linalg import svds
# import numpy as np

# # Specify the number of components (right singular vectors) you want to retain
# n_components = 150  # Decrease the number of components
# start_time = time.time()
# # Perform sparse SVD on the large sparse matrix X
# _, _, Vt = svds(multi_train_sparse_matrix, k=n_components)  # X is your large sparse matrix
# #Y_transformed = Y.dot(Vt.T)
# print(f"--- {time.time() - start_time} seconds ---")
# #--- 3585.899343729019 seconds ---
# # it is way too long compared to TrancadedSVD

--- 3585.899343729019 seconds ---


In [ ]:
#checking if one of the methods works faster for SVD

In [6]:
# Our targets for multi will be inputs for citeseq. So, we need to get citeseq,
# normalize it and perform dimentionality reduction first.
# Then we apply this transformation for multi targets. And then tune and build XGBRegressor models 
# for new targets.
train_cite = pd.read_hdf(train_cite_inputs)

In [19]:
train_cite.shape

(70988, 22050)

In [7]:
start_time = time.time()
pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=200, random_state=42))])
pipeline.fit_transform(train_cite)
print(f"--- {time.time() - start_time} seconds ---")
print(pipeline.named_steps['pca'].explained_variance_ratio_.sum())
# PCA(n_components=200,
# --- 67.58247065544128 seconds ---
# 0.08761159987095953
# This is for future, if I want to improve

--- 67.58247065544128 seconds ---
0.08761159987095953


In [ ]:
# Inverse transform to get back to the original dimension
# I will need it to make a predition for submission.
original_data = pipeline.named_steps['pca'].inverse_transform(transformed_data)


In [8]:
# Reading targets for multiome and creating a dataset corresponding to the rows
# whit the cell_id multi_train_sample I have in train_multi dataset .

target = h5py.File(train_multi_targets,'r')#, mode)
cell_id = target['train_multi_targets']['axis1'][:]
# Convert rows to regular strings
cell_id = [item.decode('utf-8') for item in cell_id]
# Create a boolean mask for the current chunk
rows_mask = np.isin(cell_id, multi_train_sample)
# Use the mask to select the corresponding rows from the current chunk of data
target_multi_train = target['train_multi_targets']['block0_values'][rows_mask,:]
target_multi_train.shape

(84548, 23418)

In [9]:
# Next step is to apply pipeline to theese targets.
#target_train_200 = pipeline.transform(target_multi_train)
# ok, I wrongfully assumed that number of targets for mulitome is the same as number of features
#for citeseq. But cityseq has 22050 features, and multiome has 23418 targets.
# So, there will be independent pipelines for dimentiality reduction for them.
start_time = time.time()
pipeline_target_200 = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=200, random_state=42))])
target_multi_train_200 = pipeline_target_200.fit_transform(target_multi_train)
print(f"--- {time.time() - start_time} seconds ---")
print(pipeline_target_200.named_steps['pca'].explained_variance_ratio_.sum())

--- 86.16162371635437 seconds ---
0.07105860961131216


In [36]:
target_multi_train_200.shape

(84548, 200)

In [10]:
#Read dataset for test_multiome and conver it to sparse matrix
multi_test_sparse_matrix = sparse_matrix_from_file( train_multi_inputs,'train_multi_inputs', multi_test_sample)

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 39000
39000 40000
40000 41000
41000 42000
42000 43000
43000 44000
44000 45000
45000 46000
46000 47000
47000 48000
48000 49000
49000 50000
50000 51000
51000 52000
52000 53000
53000 54000
54000 55000
55000 56000
56000 57000
57000 58000
58000 59000
59000 60000
60000 61000
61000 62000
62000 63000
63000 64000
64000 65000
65000 66000
66000 67000
67000 68000
68000 69000
69000 70000
70000 71000
71000 72000
72000 73000
73000 74000
74000 75000
75000 76000
76000 77000
77000 78000
78000 79000
79000 80000
80000 81000
81000 82000
82000 83000
83000 84000
84000 85000
85

In [23]:
multi_test_sparse_matrix.shape

(21394, 228942)

In [11]:
# Apply TrancateSVD to this dataset
multi_test_500 = svd_multi_500.transform(multi_test_sparse_matrix)

In [25]:
multi_test_500.shape

(21394, 500)

In [12]:
# Read targets for test_mulitome
# Reading targets for multiome and creating a dataset corresponding to the rows
# whit the cell_id multi_test_sample that I have in test_multi.

target = h5py.File(train_multi_targets,'r')#, mode)
cell_id = target['train_multi_targets']['axis1'][:]
# Convert rows to regular strings
cell_id = [item.decode('utf-8') for item in cell_id]
# Create a boolean mask for the current chunk
rows_mask_test = np.isin(cell_id, multi_test_sample)
# Use the mask to select the corresponding rows from the current chunk of data
target_multi_test = target['train_multi_targets']['block0_values'][rows_mask_test,:]
#target_train = target['train_multi_targets']['block0_values'][multi_train_sample,:]
#target_test = target['train_multi_targets']['block0_values'][multi_test_sample,:]
target_multi_test.shape

(21394, 23418)

In [21]:
sum(rows_mask_test)

21394

In [13]:
# Apply pipeline to this dataset
target_multi_test_200 = pipeline_target_200.transform(target_multi_test)

In [37]:
target_multi_test_200.shape

(21394, 200)

In [33]:
# Find the best parameters for XGBRegressor for target 1.
# Open question. Should we search for best parameters for every target or the set of the parameters 
# will be the same???
# build a model on train_multi
# test on test_multi
start_time = time.time()
# Create an instance of XGBRegressor
xgb_model = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based')
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2, random_state=3)

# Define the parameter grid for GridSearchCV
param_grid = {
    #'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.1, 0.5, 1, 10],
    'min_child_weight': [1, 5, 100],
    #'subsample': [0.7, 0.8, 0.9],
    #'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [1, 5, 100],
    'reg_lambda': [1, 5, 100]
}

# Create a GridSearchCV instance
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, scoring='neg_mean_squared_error',verbose = 3)

# Fit the GridSearchCV instance on the data
grid_search.fit(multi_train_500, target_multi_train_200[:,0])

# Print the best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 1/5] END learning_rate=0.1, max_depth=3, min_child_weight=1, reg_alpha=1, reg_lambda=1;, score=-131.048 total time=   2.6s
[CV 2/5] END learning_rate=0.1, max_depth=3, min_child_weight=1, reg_alpha=1, reg_lambda=1;, score=-140.856 total time=   1.9s
[CV 3/5] END learning_rate=0.1, max_depth=3, min_child_weight=1, reg_alpha=1, reg_lambda=1;, score=-138.577 total time=   1.9s
[CV 4/5] END learning_rate=0.1, max_depth=3, min_child_weight=1, reg_alpha=1, reg_lambda=1;, score=-173.983 total time=   1.9s
[CV 5/5] END learning_rate=0.1, max_depth=3, min_child_weight=1, reg_alpha=1, reg_lambda=1;, score=-156.741 total time=   1.9s
[CV 1/5] END learning_rate=0.1, max_depth=3, min_child_weight=1, reg_alpha=1, reg_lambda=5;, score=-132.562 total time=   1.9s
[CV 2/5] END learning_rate=0.1, max_depth=3, min_child_weight=1, reg_alpha=1, reg_lambda=5;, score=-142.010 total time=   1.9s
[CV 3/5] END learning_rate=0.1, max_depth=3, mi

/home/skovtun/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:478: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)


[CV 1/5] END learning_rate=10, max_depth=3, min_child_weight=1, reg_alpha=1, reg_lambda=1;, score=-inf total time=   1.8s


/home/skovtun/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:478: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)


[CV 2/5] END learning_rate=10, max_depth=3, min_child_weight=1, reg_alpha=1, reg_lambda=1;, score=-inf total time=   1.8s


...

[CV 3/5] END learning_rate=10, max_depth=9, min_child_weight=100, reg_alpha=100, reg_lambda=100;, score=-inf total time=   2.8s


/home/skovtun/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:478: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)


[CV 4/5] END learning_rate=10, max_depth=9, min_child_weight=100, reg_alpha=100, reg_lambda=100;, score=-inf total time=   2.6s


/home/skovtun/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:478: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/home/skovtun/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py:976: UserWarning: One or more of the test scores are non-finite: [-148.24100342 -149.29172363 -149.46287231 -148.83173523 -149.3417511
 -149.45248413 -149.10612488 -149.37667542 -149.93776245 -148.23863525
 -149.29287415 -149.4627594  -148.70904846 -149.3416687  -149.45248413
 -149.10612488 -149.37619324 -149.93775024 -148.85957336 -148.95548401
 -149.5357605  -148.89323425 -148.95905457 -149.31433411 -148.91678162
 -149.30764771 -149.64533997 -135.21628265 -135.09506073 -136.34128113
 -134.75638123 -135.30539703 -136.20346222 -135.61239166 -135.64543915
 -136.2148056  -135.00627594 -135.26484375 -136.15184784 -134.71492462
 -135.27619171 -136.3336319  -135.49360046 -135.58504028 -136.2155

[CV 5/5] END learning_rate=10, max_depth=9, min_child_weight=100, reg_alpha=100, reg_lambda=100;, score=-inf total time=   2.6s
Best parameters: {'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 100, 'reg_alpha': 5, 'reg_lambda': 5}


In [36]:
print(len(y_pred_test),len(target_multi_test_200[:,0]))

10983 21394


In [31]:
#Best parameters: {'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 100, 'reg_alpha': 5, 'reg_lambda': 5}
# Get the best model from the grid search
best_model = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based',
                         learning_rate = 0.1, max_depth = 9, min_child_weight = 100, reg_alpha = 5,
                          reg_lambda = 5)

# Use the best model to make predictions on the train set
best_model.fit(multi_train_500,target_multi_train_200[:,0] ) 
y_pred = best_model.predict(multi_train_500) 
# Calculate the mean squared error on the train set
pe = scipy.stats.pearsonr(target_multi_train_200[:,0], y_pred)
print("Pearson Error on train set:", pe)
# Use the best model to make predictions on the test set
y_pred_test = best_model.predict(multi_test_500) 
# Calculate the mean squared error on the train set
pe_test = scipy.stats.pearsonr(target_multi_test_200[:,0], y_pred_test)
print("Pearson Error on test set:", pe_test)

Pearson Error on train set: PearsonRResult(statistic=0.8959998662929849, pvalue=0.0)
Pearson Error on test set: PearsonRResult(statistic=0.7805675900860645, pvalue=0.0)


In [18]:
# Find the best parameters for XGBRegressor for target 1.
# Open question. Should we search for best parameters for every target or the set of the parameters 
# will be the same???
# Running grid search for the second target
start_time = time.time()
# Create an instance of XGBRegressor
xgb_model = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based')
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2, random_state=3)

# Define the parameter grid for GridSearchCV
param_grid = {
    #'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.1, 0.5],
    'min_child_weight': [5, 100],
    #'subsample': [0.7, 0.8, 0.9],
    #'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [ 5, 100],
    'reg_lambda': [ 5, 100]
}

# Create a GridSearchCV instance
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, scoring='r2',verbose = 3)

# Fit the GridSearchCV instance on the data
grid_search.fit(multi_train_500, target_multi_train_200[:,1])

# Print the best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV 1/5] END learning_rate=0.1, max_depth=3, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.496 total time=   2.0s
[CV 2/5] END learning_rate=0.1, max_depth=3, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.641 total time=   1.9s
[CV 3/5] END learning_rate=0.1, max_depth=3, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.538 total time=   1.9s
[CV 4/5] END learning_rate=0.1, max_depth=3, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.551 total time=   1.9s
[CV 5/5] END learning_rate=0.1, max_depth=3, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.059 total time=   1.9s
[CV 1/5] END learning_rate=0.1, max_depth=3, min_child_weight=5, reg_alpha=5, reg_lambda=100;, score=0.493 total time=   1.9s
[CV 2/5] END learning_rate=0.1, max_depth=3, min_child_weight=5, reg_alpha=5, reg_lambda=100;, score=0.638 total time=   1.9s
[CV 3/5] END learning_rate=0.1, max_depth=3, min_child_weight=5, r

In [ ]:
# let's compare performance of the model for this parameters vs chosen for the previous model

Best parameters: {'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 100, 'reg_alpha': 5, 'reg_lambda': 1}

In [33]:
best_model_0 = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based',
                         learning_rate = 0.1, max_depth = 9, min_child_weight = 100, reg_alpha = 5,
                          reg_lambda = 5)

# Use the best model to make predictions on the train set
best_model_0.fit(multi_train_500,target_multi_train_200[:,1] ) 
y_pred = best_model_0.predict(multi_train_500) 
# Calculate the mean squared error on the train set
pe = scipy.stats.pearsonr(target_multi_train_200[:,1], y_pred)
print("Pearson Error on train set:", pe)
# Use the best model to make predictions on the test set
y_pred_test = best_model_0.predict(multi_test_500) 
# Calculate the mean squared error on the train set
pe_test = scipy.stats.pearsonr(target_multi_test_200[:,1], y_pred_test)
print("Pearson Error on test set:", pe_test)

Pearson Error on train set: PearsonRResult(statistic=0.9068645128190986, pvalue=0.0)
Pearson Error on test set: PearsonRResult(statistic=0.8306190647295726, pvalue=0.0)


In [34]:
best_model_1 = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based',
                         learning_rate = 0.1, max_depth = 9, min_child_weight = 100, reg_alpha = 5,
                          reg_lambda = 1)

# Use the best model to make predictions on the train set
best_model_1.fit(multi_train_500,target_multi_train_200[:,1] ) 
y_pred = best_model_1.predict(multi_train_500) 
# Calculate the mean squared error on the train set
pe = scipy.stats.pearsonr(target_multi_train_200[:,1], y_pred)
print("Pearson Error on train set:", pe)
# Use the best model to make predictions on the test set
y_pred_test = best_model_1.predict(multi_test_500) 
# Calculate the mean squared error on the train set
pe_test = scipy.stats.pearsonr(target_multi_test_200[:,1], y_pred_test)
print("Pearson Error on test set:", pe_test)

Pearson Error on train set: PearsonRResult(statistic=0.9073754943957013, pvalue=0.0)
Pearson Error on test set: PearsonRResult(statistic=0.8294296659199507, pvalue=0.0)


For second target value grid search didn't really change anything.
Let's do grid search for the last target variable, the weakest one.


In [20]:
best_model_1r = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based',
                         learning_rate = 0.1, max_depth = 9, min_child_weight = 100, reg_alpha = 5,
                          reg_lambda = 5)

# Use the best model to make predictions on the train set
best_model_1r.fit(multi_train_500,target_multi_train_200[:,1] ) 
y_pred = best_model_1r.predict(multi_train_500) 
# Calculate the mean squared error on the train set
pe = scipy.stats.pearsonr(target_multi_train_200[:,1], y_pred)
print("Pearson Error on train set:", pe)
# Use the best model to make predictions on the test set
y_pred_test = best_model_1r.predict(multi_test_500) 
# Calculate the mean squared error on the train set
pe_test = scipy.stats.pearsonr(target_multi_test_200[:,1], y_pred_test)
print("Pearson Error on test set:", pe_test)
# learning_rate = 0.1, max_depth = 9, min_child_weight = 100, reg_alpha = 5,
#                           reg_lambda = 5
# Pearson Error on train set: PearsonRResult(statistic=0.9068645128190986, pvalue=0.0)
# Pearson Error on test set: PearsonRResult(statistic=0.8306190647295726, pvalue=0.0)

Pearson Error on train set: PearsonRResult(statistic=0.9073754943957013, pvalue=0.0)
Pearson Error on test set: PearsonRResult(statistic=0.8294296659199507, pvalue=0.0)


In [14]:
# Find the best parameters for XGBRegressor for target 1.
# Open question. Should we search for best parameters for every target or the set of the parameters 
# will be the same???
# Running grid search for the second target
start_time = time.time()
# Create an instance of XGBRegressor
xgb_model = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based')
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2, random_state=3)

# Define the parameter grid for GridSearchCV
param_grid = {
    #'n_estimators': [100, 200, 300],
    'max_depth': [ 5, 7, 9],
    'learning_rate': [0.1, 0.5, 1],
    'min_child_weight': [ 5, 100],
    #'subsample': [0.7, 0.8, 0.9],
    #'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [1, 5, 100],
    'reg_lambda': [1, 5, 100]
}

# Create a GridSearchCV instance
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, scoring='neg_mean_squared_error',verbose = 3)

# Fit the GridSearchCV instance on the data
grid_search.fit(multi_train_500, target_multi_train_200[:,199])

# Print the best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV 1/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=1, reg_lambda=1;, score=-1.782 total time=   2.9s
[CV 2/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=1, reg_lambda=1;, score=-1.904 total time=   2.4s
[CV 3/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=1, reg_lambda=1;, score=-2.046 total time=   2.4s
[CV 4/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=1, reg_lambda=1;, score=-2.163 total time=   2.4s
[CV 5/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=1, reg_lambda=1;, score=-2.052 total time=   2.5s
[CV 1/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=1, reg_lambda=5;, score=-1.781 total time=   2.4s
[CV 2/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=1, reg_lambda=5;, score=-1.905 total time=   2.4s
[CV 3/5] END learning_rate=0.1, max_depth=5, min_child_weight=

In [ ]:
Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 100, 'reg_alpha': 100, 'reg_lambda': 100}

In [15]:
best_model_199 = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based',
                         learning_rate = 0.1, max_depth = 5, min_child_weight = 100, reg_alpha = 100,
                          reg_lambda = 100)

# Use the best model to make predictions on the train set
best_model_199.fit(multi_train_500,target_multi_train_200[:,199] ) 
y_pred = best_model_199.predict(multi_train_500) 
# Calculate the mean squared error on the train set
pe = scipy.stats.pearsonr(target_multi_train_200[:,199], y_pred)
print("Pearson Error on train set:", pe)
# Use the best model to make predictions on the test set
y_pred_test = best_model_199.predict(multi_test_500) 
# Calculate the mean squared error on the train set
pe_test = scipy.stats.pearsonr(target_multi_test_200[:,199], y_pred_test)
print("Pearson Error on test set:", pe_test)
#
#Pearson Error on train set: PearsonRResult(statistic=0.4243820788326715, pvalue=0.0)
#Pearson Error on test set: PearsonRResult(statistic=0.00708862640410345, pvalue=0.299836670722249)

Pearson Error on train set: PearsonRResult(statistic=0.4243820788326715, pvalue=0.0)
Pearson Error on test set: PearsonRResult(statistic=0.00708862640410345, pvalue=0.299836670722249)


In [25]:
best_model_0199 = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based',
                         learning_rate = 0.1, max_depth = 9, min_child_weight = 100, reg_alpha = 5,
                          reg_lambda = 5)

# Use the best model to make predictions on the train set
best_model_0199.fit(multi_train_500,target_multi_train_200[:,199] ) 
y_pred = best_model_0199.predict(multi_train_500) 
# Calculate the mean squared error on the train set
pe = scipy.stats.pearsonr(target_multi_train_200[:,199], y_pred)
print("Pearson Error on train set:", pe)
# Use the best model to make predictions on the test set
y_pred_test = best_model_0199.predict(multi_test_500) 
# Calculate the mean squared error on the train set
pe_test = scipy.stats.pearsonr(target_multi_test_200[:,199], y_pred_test)
print("Pearson Error on test set:", pe_test)
#
# Pearson Error on train set: PearsonRResult(statistic=0.8224463817351744, pvalue=0.0)
# Pearson Error on test set: PearsonRResult(statistic=0.010195725056317023, pvalue=0.13589626244342787)

Pearson Error on train set: PearsonRResult(statistic=0.8224463817351744, pvalue=0.0)
Pearson Error on test set: PearsonRResult(statistic=0.010195725056317023, pvalue=0.13589626244342787)


In [ ]:
# Repeat for all other 199 targets.
pearson_list = []
model_0 = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based',
                         learning_rate = 0.1, max_depth = 9, min_child_weight = 100, reg_alpha = 5,
                          reg_lambda = 5)

# Use the best model to make predictions on the train set
model_0.fit(multi_train_500,target_multi_train_200[:,0] ) 
y_pred = model_0.predict(multi_train_500) 
# Calculate the mean squared error on the train set
pearson_list.append = scipy.stats.pearsonr(target_multi_train_200[:,0], y_pred)

In [27]:
models_list = []
pearsons_coeff_list = []

for i in range(200):
    model = XGBRegressor(
        n_jobs=-1,
        tree_method="gpu_hist",
        sampling_method='gradient_based',
        learning_rate=0.1,
        max_depth=9,
        min_child_weight=100,
        reg_alpha=5,
        reg_lambda=5
    )

    model.fit(multi_train_500, target_multi_train_200[:, i])
   
    y_pred = model.predict(multi_train_500)

    pearson_coefficient, _ = scipy.stats.pearsonr(target_multi_train_200[:, i], y_pred)

    models_list.append(model)
    pearsons_coeff_list.append(pearson_coefficient)

     #joblib.dump(model, f'model_{i}.joblib')



In [30]:
#let's apply these models to see pearson coefficients on the test set.
pearsons_coeff_test = []

for i in range(200):
    model = models_list[i]
    y_pred = model.predict(multi_test_500)
   
    pearson_coefficient, _ = scipy.stats.pearsonr(target_multi_test_200[:, i], y_pred)

    pearsons_coeff_test.append(pearson_coefficient)


In [31]:
print(pearsons_coeff_test)

[0.7805675900860645, 0.8306190647295726, 0.9678133633958059, 0.9325118463585678, 0.9112741730274543, 0.8955094539524489, 0.851513609936407, 0.8759809623843277, 0.8247752428261416, 0.7762159548563389, 0.70285870052529, 0.8480778704845742, 0.758271228963159, 0.7122943105465473, 0.825085433708977, 0.8149800400810056, 0.8077221818950991, 0.6805292143118328, 0.7132296629471466, 0.7804285787711577, 0.796358533840319, 0.7733660241623233, 0.7092600115711553, 0.5650347812800365, 0.6834080648664121, 0.7175547416429876, 0.5608989203967208, 0.551850050611929, 0.5767895990855253, 0.6299356430328185, 0.5623565260398515, 0.5326110735468638, 0.6372237145975664, 0.555938567286991, 0.6294570960235336, 0.5512302324651059, 0.5946201338531802, 0.5774966370585057, 0.5199250716716715, 0.5147361056592596, 0.4812996944316733, 0.5276847286205476, 0.5250870495880868, 0.4863689427066077, 0.50728679198203, 0.533697497939536, 0.47558927976042176, 0.5239602956326123, 0.4573915779269254, 0.4544063097136913, 0.4852071

In [44]:
# hypothesis is number of non null targets is different in test and train.
#let's check it
np.count_nonzero(target_multi_test_200, axis=0)
#no it's not the case. All of them are not null

array([21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394, 21394,
       21394, 21394,

In [49]:
np.min(target_multi_train_200, axis=0)

array([-61.891926 , -49.8933   , -18.283407 , -32.847282 , -32.787468 ,
       -22.86148  , -39.529118 , -18.5836   , -22.095095 , -22.735594 ,
       -16.889275 , -17.558521 , -17.100046 , -16.760113 , -13.729232 ,
       -18.396639 , -14.962747 , -15.652809 , -12.71621  , -13.198263 ,
       -21.644892 , -13.482657 , -19.517864 , -20.151182 , -15.463521 ,
       -22.655434 , -17.936625 , -12.909284 , -17.345276 , -26.279573 ,
       -13.401073 , -17.784204 , -14.28224  , -28.147425 , -18.393442 ,
       -24.201288 , -18.563536 , -25.513939 , -15.021814 , -21.21787  ,
       -24.001635 , -16.1148   , -18.731619 , -29.042685 , -18.533878 ,
       -14.730201 , -18.342525 , -20.40967  , -26.649227 , -29.938318 ,
       -27.037031 , -47.013844 , -14.503616 , -16.830393 , -12.997987 ,
       -11.787492 , -25.863493 , -28.19393  , -13.93829  , -11.188606 ,
       -24.506779 , -16.106934 , -19.482208 , -17.093868 , -18.185993 ,
       -13.375187 , -13.139248 , -23.475487 , -24.319807 , -13.2

In [48]:
np.min(target_multi_test_200, axis=0)

array([-58.965523 , -46.24764  , -17.676657 , -31.865873 , -29.653738 ,
       -22.150557 , -35.121716 , -14.49214  , -21.587835 , -18.077238 ,
       -17.673483 , -16.30125  , -16.490816 , -14.115557 , -11.8856325,
       -13.564076 , -13.560971 , -14.701867 , -11.359819 , -11.292692 ,
       -20.228369 , -10.975441 , -17.523085 , -15.260042 , -11.775464 ,
       -17.800087 , -14.176069 , -10.087726 , -15.649393 , -28.472546 ,
        -8.206872 , -13.846617 , -11.599933 , -14.041894 , -12.473182 ,
       -21.567282 , -16.053545 , -26.34353  ,  -9.2088995, -15.686449 ,
       -16.932533 ,  -8.664751 , -10.161431 ,  -9.077582 ,  -8.406501 ,
       -11.278567 ,  -8.398158 ,  -8.6635   ,  -8.41472  , -16.366436 ,
       -15.483231 , -16.818474 ,  -6.89514  , -11.55619  ,  -8.341601 ,
        -8.080852 , -10.852733 , -10.531212 , -12.479444 ,  -6.474294 ,
       -12.571712 ,  -9.319707 , -12.541662 ,  -9.379947 , -12.075114 ,
        -6.7579365,  -7.025849 ,  -8.810884 , -13.009705 ,  -6.5

In [50]:
# I have different means for train and test targets despite my effort to make train and test sets
# balanced based on phenotype data. So, I am going to merge them togeather and build my 200 models
# on the whole set.
target_multi_200 = np.concatenate((target_multi_train_200, target_multi_test_200), axis=0)
multi_500 = np.concatenate((multi_train_500, multi_test_500), axis=0)

In [52]:
# Find the best parameters for XGBRegressor for target 1.
# Open question. Should we search for best parameters for every target or the set of the parameters 
# will be the same???
# Running grid search for the second target
start_time = time.time()
# Create an instance of XGBRegressor
xgb_model = XGBRegressor(n_jobs=-1,tree_method="gpu_hist", sampling_method='gradient_based')
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2, random_state=3)

# Define the parameter grid for GridSearchCV
param_grid = {
    #'n_estimators': [100, 200, 300],
    'max_depth': [5, 9],
    'learning_rate': [0.1, 0.5],
    'min_child_weight': [5, 100],
    #'subsample': [0.7, 0.8, 0.9],
    #'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [ 5, 100],
    'reg_lambda': [ 5, 100]
}

# Create a GridSearchCV instance
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, scoring='r2',verbose = 3)

# Fit the GridSearchCV instance on the data
grid_search.fit(multi_500, target_multi_200[:,1])

# Print the best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.613 total time=   3.0s
[CV 2/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.688 total time=   3.0s
[CV 3/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.617 total time=   3.0s
[CV 4/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.402 total time=   3.0s
[CV 5/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=5, reg_lambda=5;, score=0.651 total time=   3.0s
[CV 1/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=5, reg_lambda=100;, score=0.611 total time=   3.0s
[CV 2/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, reg_alpha=5, reg_lambda=100;, score=0.686 total time=   3.0s
[CV 3/5] END learning_rate=0.1, max_depth=5, min_child_weight=5, r

In [53]:
models_list = []
pearsons_coeff_list = []

for i in range(200):
    model = XGBRegressor(
        n_jobs=-1,
        tree_method="gpu_hist",
        sampling_method='gradient_based',
        learning_rate=0.1,
        max_depth=9,
        min_child_weight=100,
        reg_alpha=5,
        reg_lambda=5
    )

    model.fit(multi_500, target_multi_200[:, i])
   
    y_pred = model.predict(multi_500)

    pearson_coefficient, _ = scipy.stats.pearsonr(target_multi_200[:, i], y_pred)

    models_list.append(model)
    pearsons_coeff_list.append(pearson_coefficient)

     #joblib.dump(model, f'model_{i}.joblib')

In [57]:
# now we can get results for kaggle test data.
test_multi = sparse_matrix_from_file( test_multi_inputs,'test_multi_inputs', [])

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 39000
39000 40000
40000 41000
41000 42000
42000 43000
43000 44000
44000 45000
45000 46000
46000 47000
47000 48000
48000 49000
49000 50000
50000 51000
51000 52000
52000 53000
53000 54000
54000 55000
55000 56000


In [58]:
test_multi.shape

(55935, 228942)

In [59]:
test_multi_500 = svd_multi_500.transform(test_multi)

In [60]:
test_multi_500_traget_200 = np.empty((len(test_multi_500), 0))

# Iterate over the models and generate predictions
for i in range(200):
    model = models_list[i]
    y_pred = model.predict(test_multi_500)
    
    # Add the predictions as a new column in the 2D array
    test_multi_500_traget_200 = np.column_stack((test_multi_500_traget_200, y_pred))

In [61]:
test_multi_500_traget_200.shape

(55935, 200)

In [63]:
test_multi_500_traget_scaled = pipeline_target_200.named_steps['pca'].inverse_transform(test_multi_500_traget_200)
test_multi_500_traget = pipeline_target_200.named_steps['scaling'].inverse_transform(test_multi_500_traget_scaled)

In [64]:
test_multi_500_traget.shape

(55935, 23418)

In [73]:
with h5py.File(test_multi_inputs, 'r') as h5file:
        rows = h5file['test_multi_inputs']['axis1'][:]
        # Convert rows to regular strings
        rows_utf_8 = [item.decode('utf-8') for item in rows]

In [81]:
with h5py.File(train_multi_targets, 'r') as h5file:
        columns = h5file['train_multi_targets']['axis0'][:]
        columns_utf_8 = [item.decode('utf-8') for item in columns]

In [ ]:
sample_submission = os.path.join(data_path,"sample_submission.csv")
evaluation_ids = os.path.join(data_path,"evaluation_ids.csv")

In [67]:
sample_submission_df = pd.read_csv(sample_submission)

In [68]:
sample_submission_df.head()

,row_id,target
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [108]:
# Reading evaluation_ids.csv and creating mapping for every cell_id and gene_id,
# where it's values got replaced by position number to be able to find the correspondent target value
evaluation_ids_df = pd.read_csv(evaluation_ids)
value_to_position_cell_id = {value: index for index, value in enumerate(rows_utf_8)}
value_to_position_gene_id = {value: index for index, value in enumerate(columns_utf_8)}
evaluation_ids_df['cell_id'] = evaluation_ids_df['cell_id'].map(value_to_position_cell_id).fillna(-1).astype(int)
evaluation_ids_df['gene_id'] = evaluation_ids_df['gene_id'].map(value_to_position_gene_id).fillna(-1).astype(int)
evaluation_ids_df_multi = evaluation_ids_df[evaluation_ids_df['cell_id'] != -1]

In [109]:
evaluation_ids_df_multi.head()


,row_id,cell_id,gene_id
6812820,6812820,11119,20687
6812821,6812821,11119,15183
6812822,6812822,11119,17190
6812823,6812823,11119,6387
6812824,6812824,11119,11320


In [99]:
evaluation_ids_df_multi.shape

(58931360, 3)

In [110]:
# Adding target value from the test_multi_500_traget to the evaluation_ids
def extract_value(row):
    row_num = row['cell_id']
    col_num = row['gene_id']
    try:
        value = test_multi_500_traget[row_num, col_num]
    except IndexError:
        value = np.nan
    
    return value
    
evaluation_ids_df_multi['Value'] = evaluation_ids_df_multi.apply(extract_value, axis=1)

/tmp/ipykernel_2413/3709660701.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_ids_df_multi['Value'] = evaluation_ids_df_multi.apply(extract_value, axis=1)


In [111]:
evaluation_ids_df_multi.head()

,row_id,cell_id,gene_id,Value
6812820,6812820,11119,20687,0.000888
6812821,6812821,11119,15183,10.283843
6812822,6812822,11119,17190,0.984811
6812823,6812823,11119,6387,1.083188
6812824,6812824,11119,11320,0.002385


In [112]:
evaluation_ids_df_multi = evaluation_ids_df_multi.rename(columns={'Value': 'target'})


In [116]:
submission_multi = evaluation_ids_df_multi[['row_id','target']].reset_index(drop=True)
submission_multi.head()

,row_id,target
0,6812820,0.000888
1,6812821,10.283843
2,6812822,0.984811
3,6812823,1.083188
4,6812824,0.002385


In [117]:
submitssion_multi_path = os.path.join(data_path,"submission_multi.csv")
submission_multi.to_csv(submitssion_multi_path, index=False)